<h1>AI in Fact and Fiction - Summer 2020</h1>
<h2>Public Emotions - Creating Training Dataset</h2>

<!-- <p><a href="">Run this notebook on collab.</a></p> -->

<hr>

<h3>Preparation</h3>

Import the following libraries that you will use for this code:

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import csv
import calendar

Upload the jsonl file and csv file needed for Tweet conversion.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving corona_tweets_95.jsonl to corona_tweets_95.jsonl


<h2 id="Reading_Data">Reading Data from Imported Files</h2>

View data in csv file to check for delimiter and help with parsing.

In [ ]:
with open('corona_tweets_95.csv', 'r') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row)
        print(row[0])

        break #I put this here just to see if things are being read. The csv file is really big, dont want to print it all

['1274559089041907712', '0.8']
1274559089041907712


Get data from jsonl file and check to see if getting the data succeeded.

In [ ]:
data = []

with open('corona_tweets_95.jsonl', 'r', encoding='utf-8') as f:
  for line in f:
    #print(line)
    data.append(json.loads(line))
    
print(data[62])

{'created_at': 'Sun Jun 21 04:50:54 +0000 2020', 'id': 1274565433216000000, 'id_str': '1274565433216000000', 'full_text': 'RT @CalEndow: #COVID19 is killing young Black Americans at an alarmingly high rate. \n\nThe pandemic has laid bare the deadly results of unde…', 'truncated': False, 'display_text_range': [0, 140], 'entities': {'hashtags': [{'text': 'COVID19', 'indices': [14, 22]}], 'symbols': [], 'user_mentions': [{'screen_name': 'CalEndow', 'name': 'California Endowment', 'id': 132254256, 'id_str': '132254256', 'indices': [3, 12]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 544973361, 'id_str': '544973361', 'name': 'Shweta', 'screen_name': 'OodLittleThing', 'location': 'North. ', 'description': '27. English Teacher. Somewhat writerly. DEFINIT

<h2 id="Convert_Data">Converting Data into Dataframes</h2>

Get data from imported csv file. Convert the csv data gathered into a Dataframe with the proper type values.

In [ ]:
csv_data = []

with open('lab4.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
      csv_data.append([])
      for item in row:
          csv_data[-1].append(item)

df_csv = pd.DataFrame(csv_data, columns=['ID','Rating'])
df_csv['ID'] = df_csv['ID'].astype(int)
df_csv['Rating'] = df_csv['Rating'].astype(float)

df_csv

,ID,Rating
0,1274559089041907712,0.800000
1,1274559089209638912,-1.000000
2,1274559089570349056,0.136364
3,1274559089696219136,0.100000
4,1274559089633193988,-0.227778
...,...,...
3352638,1274921323509080065,-0.187500
3352639,1274921323593109505,-0.275000
3352640,1274921323693699073,-0.200000
3352641,1274921323823804422,-0.900000


Get specific data columns from jsonl file and convert them into a Dataframe.

Only take tweets that are in English and remove any rows that have missing information.

In [ ]:
df = pd.DataFrame()

db_data = []
db_cols = ['id','full_text', 'created_at']
for d in data:
  if (d['lang'] == 'en'):
    db_data.append([])
    for col in db_cols:
      db_data[-1].append(d.get(col, float('nan')))

df = pd.DataFrame(db_data, columns=['ID','Tweets','Time'])
df = df.dropna()
df

,ID,Tweets,Time
0,1274559273830400000,RT @SheaSerrano: trump supporters are so fucki...,Sun Jun 21 04:26:25 +0000 2020
1,1274559347566080000,RT @DrRobDavidson: It’s clear that @realDonald...,Sun Jun 21 04:26:43 +0000 2020
2,1274559381854720000,This is literally the dumbest thing I have eve...,Sun Jun 21 04:26:51 +0000 2020
3,1274559374036480000,RT @IHME_UW: Explore #COVID19 projections of h...,Sun Jun 21 04:26:49 +0000 2020
4,1274559383779840000,RT @perlmutations: Trump’s Tulsa Rally require...,Sun Jun 21 04:26:51 +0000 2020
...,...,...,...
970,1274691318824960000,RT @jonfavs: This is an entire speech about al...,Sun Jun 21 13:11:07 +0000 2020
971,1274691408998400000,RT @bgluckman: I can't believe people are stil...,Sun Jun 21 13:11:29 +0000 2020
972,1274691662535680000,"""TFL AND LONDON MAYORS INCREASE AND CHANGES TO...",Sun Jun 21 13:12:29 +0000 2020
973,1274691912860160000,"RT @TimMurtaugh: This, obviously, is fake, and...",Sun Jun 21 13:13:29 +0000 2020


<h2 id="Date">Date Time Conversion</h2>

Get properly formated date information to be used for the AI model.

In [ ]:
df['Month'] = pd.DatetimeIndex(df['Time']).month
df['Month'] = pd.to_datetime(df['Month'], format='%m').dt.month_name()

df

,ID,Tweets,Time,Month
0,1274559273830400000,RT @SheaSerrano: trump supporters are so fucki...,Sun Jun 21 04:26:25 +0000 2020,June
1,1274559347566080000,RT @DrRobDavidson: It’s clear that @realDonald...,Sun Jun 21 04:26:43 +0000 2020,June
2,1274559381854720000,This is literally the dumbest thing I have eve...,Sun Jun 21 04:26:51 +0000 2020,June
3,1274559374036480000,RT @IHME_UW: Explore #COVID19 projections of h...,Sun Jun 21 04:26:49 +0000 2020,June
4,1274559383779840000,RT @perlmutations: Trump’s Tulsa Rally require...,Sun Jun 21 04:26:51 +0000 2020,June
...,...,...,...,...
970,1274691318824960000,RT @jonfavs: This is an entire speech about al...,Sun Jun 21 13:11:07 +0000 2020,June
971,1274691408998400000,RT @bgluckman: I can't believe people are stil...,Sun Jun 21 13:11:29 +0000 2020,June
972,1274691662535680000,"""TFL AND LONDON MAYORS INCREASE AND CHANGES TO...",Sun Jun 21 13:12:29 +0000 2020,June
973,1274691912860160000,"RT @TimMurtaugh: This, obviously, is fake, and...",Sun Jun 21 13:13:29 +0000 2020,June


<h2 id="Clean_Tweet">Clean Tweet Messages</h2>

Remove any unnecessary text from the tweet that should not affect the sentiment analysis.

This is a helper function to clean the tweet.

In [ ]:
# Clean tweet text

# Function to clean the tweets
def cleanTweet(text):
  text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
  text = re.sub('_[A-Za-z0–9]+', '', text) #Removing _words
  text = re.sub('\n', '', text) #Removing _words
  text = re.sub('#', '', text) # Removing '#' hash tag
  text = re.sub('RT[\s]+', '', text) # Removing RT
  text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
  
  return text

df['Tweets'] = df['Tweets'].apply(cleanTweet)

# Remove certain punctuations
df['Tweets'] = df['Tweets'].str.replace(':', '')

df['ID'] = df['ID'].astype(int)

df

,ID,Tweets,Time,Month
0,1274559273830400000,trump supporters are so fucking dumb that eve...,Sun Jun 21 04:26:25 +0000 2020,June
1,1274559347566080000,It’s clear that doesn’t have our backs in th...,Sun Jun 21 04:26:43 +0000 2020,June
2,1274559381854720000,This is literally the dumbest thing I have eve...,Sun Jun 21 04:26:51 +0000 2020,June
3,1274559374036480000,Explore COVID19 projections of hospital resou...,Sun Jun 21 04:26:49 +0000 2020,June
4,1274559383779840000,"Trump’s Tulsa Rally requires no masks, no soc...",Sun Jun 21 04:26:51 +0000 2020,June
...,...,...,...,...
970,1274691318824960000,This is an entire speech about all the ways t...,Sun Jun 21 13:11:07 +0000 2020,June
971,1274691408998400000,I can't believe people are still arguing abou...,Sun Jun 21 13:11:29 +0000 2020,June
972,1274691662535680000,"""TFL AND LONDON MAYORS INCREASE AND CHANGES TO...",Sun Jun 21 13:12:29 +0000 2020,June
973,1274691912860160000,"This, obviously, is fake, and a recurring lie...",Sun Jun 21 13:13:29 +0000 2020,June


<h2 id="Compile_Data">Compile Training Dataset</h2>


Merge data from jsonl and csv into a new Dataframe.

In [ ]:
df3 = pd.merge(df, df_csv, how ='inner', on ='ID') 
df3

,ID,Tweets,Time,Month,Rating
0,1274559273830400000,trump supporters are so fucking dumb that eve...,Sun Jun 21 04:26:25 +0000 2020,June,-0.087500
1,1274559347566080000,It’s clear that doesn’t have our backs in th...,Sun Jun 21 04:26:43 +0000 2020,June,0.100000
2,1274559381854720000,This is literally the dumbest thing I have eve...,Sun Jun 21 04:26:51 +0000 2020,June,0.000000
3,1274559374036480000,Explore COVID19 projections of hospital resou...,Sun Jun 21 04:26:49 +0000 2020,June,0.011111
4,1274559383779840000,"Trump’s Tulsa Rally requires no masks, no soc...",Sun Jun 21 04:26:51 +0000 2020,June,-0.016667
...,...,...,...,...,...
876,1274691318824960000,This is an entire speech about all the ways t...,Sun Jun 21 13:11:07 +0000 2020,June,0.031250
877,1274691408998400000,I can't believe people are still arguing abou...,Sun Jun 21 13:11:29 +0000 2020,June,0.000000
878,1274691662535680000,"""TFL AND LONDON MAYORS INCREASE AND CHANGES TO...",Sun Jun 21 13:12:29 +0000 2020,June,0.000000
879,1274691912860160000,"This, obviously, is fake, and a recurring lie...",Sun Jun 21 13:13:29 +0000 2020,June,-0.500000


Remove the tweet ID column. This data is not needed in the dataset to train the AI model.

In [ ]:
df3 = df3.drop(columns=['ID'])
df3

,Tweets,Time,Month,Rating
0,trump supporters are so fucking dumb that eve...,Sun Jun 21 04:26:25 +0000 2020,June,-0.087500
1,It’s clear that doesn’t have our backs in th...,Sun Jun 21 04:26:43 +0000 2020,June,0.100000
2,This is literally the dumbest thing I have eve...,Sun Jun 21 04:26:51 +0000 2020,June,0.000000
3,Explore COVID19 projections of hospital resou...,Sun Jun 21 04:26:49 +0000 2020,June,0.011111
4,"Trump’s Tulsa Rally requires no masks, no soc...",Sun Jun 21 04:26:51 +0000 2020,June,-0.016667
...,...,...,...,...
876,This is an entire speech about all the ways t...,Sun Jun 21 13:11:07 +0000 2020,June,0.031250
877,I can't believe people are still arguing abou...,Sun Jun 21 13:11:29 +0000 2020,June,0.000000
878,"""TFL AND LONDON MAYORS INCREASE AND CHANGES TO...",Sun Jun 21 13:12:29 +0000 2020,June,0.000000
879,"This, obviously, is fake, and a recurring lie...",Sun Jun 21 13:13:29 +0000 2020,June,-0.500000


Remove the detailed time column. This data is not needed in the dataset to train the AI model.

In [ ]:
df3 = df3.drop(columns=['Time'])
df3

,Tweets,Month,Rating
0,trump supporters are so fucking dumb that eve...,June,-0.087500
1,It’s clear that doesn’t have our backs in th...,June,0.100000
2,This is literally the dumbest thing I have eve...,June,0.000000
3,Explore COVID19 projections of hospital resou...,June,0.011111
4,"Trump’s Tulsa Rally requires no masks, no soc...",June,-0.016667
...,...,...,...
876,This is an entire speech about all the ways t...,June,0.031250
877,I can't believe people are still arguing abou...,June,0.000000
878,"""TFL AND LONDON MAYORS INCREASE AND CHANGES TO...",June,0.000000
879,"This, obviously, is fake, and a recurring lie...",June,-0.500000


<h2 id="Convert">Converting Dataframe to CSV</h2>

Remove the row labels and the column labels for a properly formated Dataframe. Convert the newly edited final Dataframe into a csv file. 

In [ ]:
from google.colab import files
df3.to_csv('test4.csv', index=False, header=False)
files.download('test4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>